# Load Filenames and Times for TEMPO Data

This notebook:
1) Pulls the filenames for all available L3 files from s3
2) Pulls the filenames for all L2 files for swaths that cross over the bounding box
3) Map L2 to L3 files
4) Calculate L3 times by taking the midpoint of the swath times (halfway between beginning of first swath and end of last swath)


In [1]:
import boto3
import xesmf as xe
import xarray as xr
import netCDF4
import pandas as pd
import argparse
import base64
import boto3
import json
import requests
import os
import platform
from subprocess import Popen
import shutil
from datetime import datetime
import re
import pandas as pd
import earthaccess
from load_credentials import *
from bbox import *

# Define event with credentials and S3 endpoint details
event = {
    's3_endpoint': 'https://data.asdc.earthdata.nasa.gov/s3credentials',  # replace with actual endpoint
    'edl_username': 'amanda.murray19',  # replace with your EDL username
    'edl_password': 'Sat_modeling_berk2024',  # replace with your EDL password
    'bucket_name': 'asdc-prod-protected/TEMPO/TEMPO_O3TOT_L3_V03'  # replace with your bucket name
}

## Get all L3 file names from s3

In [ ]:
# Remove .netrc file if it exists
netrc_path = os.path.expanduser('~/.netrc')  # Expands to the user's home directory
if os.path.exists(netrc_path):
    print(".netrc file exists.")
    os.remove(netrc_path)
    print(".netrc file has been removed.")
else:
    print(".netrc file does not exist.")# Retrieve credentials

creds = retrieve_credentials(event)

# Use the credentials to access the S3 bucket
client = boto3.client('s3',
    aws_access_key_id=creds["accessKeyId"],
    aws_secret_access_key=creds["secretAccessKey"],
    aws_session_token=creds["sessionToken"])

bucket_name = 'asdc-prod-protected'
prefix = 'TEMPO/TEMPO_O3TOT_L3_V03/'

client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# List objects in the bucket
response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
tempo_l3_files = []

# Loop to handle pagination if there are more than 1000 objects
while response.get('Contents'):
    # Append the object keys to the list
    tempo_l3_files.extend([r["Key"] for r in response['Contents']])

    # Check if there is a next page of results
    if response.get('NextContinuationToken'):
        response = client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, ContinuationToken=response['NextContinuationToken'])
    else:
        break

# Pull only L3 files
tempo_l3_files_nc = []

for file_name in tempo_l3_files:
    if file_name.endswith('.nc'):
        tempo_l3_files_nc.append(file_name)

.netrc file does not exist.
<Response [307]>


## Get all L3 file names from s3

In [ ]:
# Write files to a text file
with open("/home/jupyter/.netrc", "w") as f:
   f.write("""machine urs.earthdata.nasa.gov
login amanda.murray19
password Sat_modeling_berk2024""")

# Establishing access to EarthData,
auth = earthaccess.login(strategy="netrc", persist=True)

# Get current date and time
current_datetime = datetime.now()
formatted_current_datetime = current_datetime.strftime('%Y-%m-%d %H:%M:%S')

#Set up collection information along with timeframe and region of interest
short_name = 'TEMPO_O3TOT_L2' # collection name to search for in the EarthData
date_start = '2022-05-01 00:00:00' # start date
date_end = formatted_current_datetime # end date
bbox = (lon_min, lat_min, lon_max, lat_max) # 1 degree bounding box around POI

# search for granules
results = earthaccess.search_data(short_name = short_name\
, temporal = (date_start, date_end)
, bounding_box = bbox)



In [ ]:
file_pattern = re.compile(r"TEMPO/TEMPO_O3TOT_L2_V\d+/[\d.]+/TEMPO_O3TOT_L2_V\d+_\d+T\d+Z_S\d+G\d+\.nc")
datetime_pattern = re.compile(r'BeginningDateTime\': \'([^\']+)\', \'EndingDateTime\': \'([^\']+)')

# Initialize a list to store the data
data = []

# Iterate through your results
for result in results:  # Adjust `results[:4]` as needed
    files = file_pattern.findall(str(result))
    datetimes = datetime_pattern.findall(str(result))

    # Ensure both files and datetimes have matches before proceeding
    if files and datetimes:
        for file, (begin, end) in zip(files, datetimes):
            # Append each record as a tuple (or dictionary)
            data.append({
                "L2_File": file,
                "BeginningDateTime": begin,
                "EndingDateTime": end
            })

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Convert BeginningDateTime and EndingDateTime columns to timestamps
df['BeginningDateTime'] = pd.to_datetime(df['BeginningDateTime'])
df['EndingDateTime'] = pd.to_datetime(df['EndingDateTime'])

# Extract the date (20230809) and file number (S011) using regular expressions
df['Date'] = df['L2_File'].str.extract(r'_(\d{8})T')
df['FileNumber'] = df['L2_File'].str.extract(r'_(S\d+)')

# Set beginning_time as the time of first swath and ending_time as end of last swath
df_grouped = df.groupby(['Date', 'FileNumber']).agg({'L2_File':'count', 'BeginningDateTime':'min','EndingDateTime':'max'}).reset_index()
# Make Time as the midpoint
df_grouped['Time'] = df_grouped['BeginningDateTime']+(df_grouped['EndingDateTime'] - df_grouped['BeginningDateTime'])/2
df_grouped['Time'] = df_grouped['Time'].round('S')

# Create a DataFrame from the list of lines
df_L3 = pd.DataFrame(tempo_l3_files_nc, columns=['FilePath'])
df_L3['Date'] = df_L3['FilePath'].str.extract(r'_(\d{8})T')
df_L3['FileNumber'] = df_L3['FilePath'].str.extract(r'_(S\d+)')

# Full dataset
full_dataset = pd.merge(df_L3, df_grouped, on=['Date', 'FileNumber'], how='inner')

In [ ]:
# Convert to central
full_dataset['timestamp_ct'] = full_dataset['Time'].dt.tz_convert('America/Chicago')
full_dataset['date_central'] = full_dataset['timestamp_ct'].dt.date

# Remove the timezone part from the string (keeping only the date and time)
full_dataset['timestamp_ct'] = full_dataset['timestamp_ct'].dt.strftime('%Y-%m-%d %H:%M:%S')
full_dataset['timestamp_ct'] = full_dataset['timestamp_ct'].str[:19]

# Convert the 'timestamp_ct' column to datetime
full_dataset['timestamp_ct'] = pd.to_datetime(full_dataset['timestamp_ct'])
full_dataset['Hour'] = full_dataset['timestamp_ct'].dt.hour
full_dataset['Minute'] = full_dataset['timestamp_ct'].dt.minute
full_dataset =full_dataset.sort_values('timestamp_ct')

In [ ]:
full_dataset.to_csv("../../data/tempo_data/o3_tempo_files_df.csv")